In [ ]:
import sympy as sp
sp.init_printing()
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
nk, nk1, nk2, mk, mk1, pk, pk1 =sp.symbols('n_k n_{k+1} n_{k+2} m_k m_{k+1} p_k p_{k+1}')

In [ ]:
cost_stage = nk1*nk + nk1*mk+pk*nk+pk*mk
cost_stage1 = nk2*nk1 + nk2*mk1+pk1*nk1+pk1*mk1

cost_both = cost_stage+cost_stage1
cost_both

# Move left

Change input

$$\tilde{p}_k =p_k+1 $$
$$\tilde{p}_{k+1}= p_{k+1} -1$$

In [ ]:
sp.simplify(cost_both.subs({pk:pk+1,pk1:pk1-1})-cost_both)


Cost if $n_{k+1}$ gets samller
$$\tilde{n}_{k+1} = _{k+1}-1$$

In [ ]:
sp.simplify(cost_both.subs({pk:pk+1,pk1:pk1-1,nk1:nk1-1})-cost_both)


# Move rigth

Change input

$$\tilde{p}_k =p_k-1 $$
$$\tilde{p}_{k+1}= p_{k+1} +1$$

In [ ]:
sp.simplify(cost_both.subs({pk:pk-1,pk1:pk1+1})-cost_both)


Cost if $n_{k+1}$ gets bigger
$$\tilde{n}_{k+1} = _{k+1}+1$$

In [ ]:
sp.simplify(cost_both.subs({pk:pk+1,pk1:pk1-1,nk1:nk1+1})-cost_both)


# Move down

Change output

$$\tilde{m}_k =m_k+1 $$
$$\tilde{m}_{k+1}= m_{k+1} -1$$

In [ ]:
sp.simplify(cost_both.subs({mk:mk+1,mk1:mk1-1})-cost_both)


Cost if $n_{k+1}$ gets smaller
$$\tilde{n}_{k+1} = _{k+1}-1$$

In [ ]:
sp.simplify(cost_both.subs({mk:mk+1,mk1:mk1-1,nk1:nk1-1})-cost_both)


# Move up

Change output

$$\tilde{m}_k =m_k-1 $$
$$\tilde{m}_{k+1}= m_{k+1} +1$$

In [ ]:
sp.simplify(cost_both.subs({mk:mk-1,mk1:mk1+1})-cost_both)


Cost if $n_{k+1}$ gets bigger
$$\tilde{n}_{k+1} = _{k+1}+1$$

In [ ]:
sp.simplify(cost_both.subs({mk:mk-1,mk1:mk1+1,nk1:nk1+1})-cost_both)


# Some helping function

Transform the sum into a polynomial

calcaultes $$\sum^N_{n=1} exp(n) $$  for $$exp = \sum_{p=0}^5 a_p n^p$$

In [ ]:
def transfrom_sum(arg,n,N,n_start=None):
    #Take a sum consisting of polynomilas in n and calcualtes the overall polynomial using the Faulhabersche_Formel
    #calcaultes $$\sum^N_{n=1} exp$$  for $$exp = \sum_{p=0}^5 a_p n^p$$
    #Parameters:
    #    arg:   expression in the sum
    #    n:     running variable
    #    N:     upper end of sum
    #returns:
    # expression 
    arg = arg.expand().collect(n)
    expression = 0
    for i in range(6):
        #faulhaber coeffs form: https://de.wikipedia.org/wiki/Faulhabersche_Formel 
        #look below for more details
        coeffs = {0:N,
              1:1/sp.S(2)*N**2+1/sp.S(2)*N,
              2:1/sp.S(3)*N**3+1/sp.S(2)*N**2+1/sp.S(6)*N,
              3:1/sp.S(4)*N**4+1/sp.S(2)*N**3+1/sp.S(4)*N**2,
              4:1/sp.S(5)*N**5+1/sp.S(2)*N**4+1/sp.S(3)*N**3-1/sp.S(30)*N,
              5:1/sp.S(6)*N**6+1/sp.S(2)*N**5+5/sp.S(12)*N**4-1/sp.S(12)*N**2}


        expression = expression + (arg.coeff(n,i))*coeffs[i] 
        if n_start:
            expression = expression - (arg.coeff(n,i))*coeffs[i].subs({N:n_start-1})
            #display((arg.coeff(n,i))*coeffs[i].subs({N:n_start-1}))
    #test for higher orders
    for i in range(6,100):
        if arg.coeff(n,i) !=0:
            raise NotImplementedError("n^"+str(i)+" is not implemented")
    return expression

# Approximations of Bounds

$$ m = N/v $$
$$ m = N/v $$

## Shapes of the Hankel matrix

height:
$$ \nu -\sum_{k`=1}^{k-1} p_{k`}= N - (k-1)p = (v-k+1)p$$
$$ \sum_{k`=1}^{k-1} m_{k`} = (k-1)m$$

## Approximate rank

$$rank(H) \approx \gamma \cdot \text{height} \cdot \text{width}$$

### Product of height and width:
$$(v-k+1)p(k-1)m$$

### Minimum of height and width (here $p=m$)
$$ \min((v-k+1)p,(k-1)m) = \min\big((v-k+1),(k-1)\big)m = \min\big(v+1-k,k-1\big)m$$

change this 
$v+1-k>k-1 \Leftrightarrow v+1>2k-1 \Leftrightarrow v+2>2k \Leftrightarrow v/2+1>k$

$$n_k = \gamma \min((v-k+1)p,(k-1)m) = \begin{cases}
    \gamma (k-1)m &\text{if } k\leq v/2+1 \\
    \gamma (v+1-k)m &\text{if } k > v/2+1 \\
\end{cases}$$




In [ ]:
# numerical test for N=100
ks = np.arange(1,100+1)
n_prod = (100-ks+1)*(ks-1)/100 #m=1 and p=1 as 100/100
n_min = np.min(np.vstack((100-ks+1,ks-1)),axis=0)
plt.plot(ks,n_prod,label="prod")
plt.plot(ks,n_min ,label="min")

In [ ]:
v,m,p,Nu = sp.symbols('v m p Nu',positive=True)
nk,nk1 = sp.symbols('n_k n_{k+1}',positive=True)
gamma = sp.symbols('gamma',positive=True)

In [ ]:
k = sp.symbols('k')
#include D for anticausal part
#cost =nk1*nk+nk1*m+p*nk+p*m
# only include D once -> divide it by 2
cost =nk1*nk+nk1*m+p*nk+0.5*p*m
cost_sum = sp.Sum(cost,(k,1,v))
cost_sum

# Product

In [ ]:
cost_prod=cost_sum.subs({nk:gamma*(v-k+1)*(k-1)*p*m,nk1:gamma*(v-k+2)*(k)*p*m})
sp.simplify(cost_prod)

In [ ]:
arg_prod = cost.subs({nk:gamma*(v-k+1)*(k-1)*p*m,nk1:gamma*(v-k+2)*(k)*p*m})
arg_prod

In [ ]:
cost_total_prod = transfrom_sum(arg_prod,k,v)
cost_total_prod

relation between $m$ and $p$ and $v$

Here a quadratic matrix:
$$p = N/v$$
$$m = N/v$$

In [ ]:
cost_total_prod=cost_total_prod.subs({p:Nu/v,m:Nu/v}).expand().collect(v)
cost_total_prod

In [ ]:
deriv = sp.diff(cost_total_prod,v)
deriv

In [ ]:
#critical_points=sp.solve((deriv*v**4).subs({Nu:100}).expand().collect(v),v)

In [ ]:
f = sp.lambdify([v,(Nu,gamma)],cost_total_prod)
vs = np.linspace(1,1000,500)
for gamma_tilde in np.logspace(-6,-0.5,10):
    costs= f(vs,(500,gamma_tilde*1./100))
    plt.plot(vs,costs,label="g="+str(gamma_tilde))
    
plt.legend()

## Now minimum

$$n_k = \gamma \min((v-k+1)p,(k-1)m) = \begin{cases}
    \gamma (k-1)m &\text{if } k\leq v/2+1 \\
    \gamma (v+1-k)m &\text{if } k > v/2+1 \\
\end{cases}$$

This gives 

$$\text{Total Cost} = \sum_{k=1}^{v/2}\text{cost}_{k} + \sum_{k=v/2+1}^{v}\text{cost}_{k}$$

For $v/2+1$ we can use either one so we can include $v/2$ in the first sum, at leasst for even $v$

In [ ]:
display(cost)
arg_min1 = cost.subs({nk:gamma*(k-1)*m,nk1:gamma*(k)*m})
arg_min2 = cost.subs({nk:gamma*(v+1-k)*m,nk1:gamma*(v-k)*m})

display(arg_min1)

cost_total_min = transfrom_sum(arg_min1,k,v/2)+transfrom_sum(arg_min2,k,v,n_start=v/2+1)
cost_total_min

In [ ]:
cost_total_min=cost_total_min.subs({p:Nu/v,m:Nu/v}).expand().collect(v)
cost_total_min

In [ ]:
N = 100
f = sp.lambdify([v,(Nu,gamma)],cost_total_min)
vs = np.arange(1,N+1)
for gamma_tilde in np.logspace(-2,0,10):
    costs= f(vs,(100,gamma_tilde))
    plt.plot(vs,costs,label="g="+str(gamma_tilde))
    
plt.legend()

In [ ]:
f(np.array([2,5,10,25,50]),(100,1))*2 #*2 for anticuasal part

### Small test of the coeffs:

Faulhaber coeffs form: https://de.wikipedia.org/wiki/Faulhabersche_Formel 
$$\sum^v_{k=1}k^p$$

In [ ]:
n,N,m=sp.symbols('n N m',postive = True)
transfrom_sum(n**3,n,N,N-1).expand()

In [ ]:
((N-1)**3+N**2).expand()

In [ ]:
n,N,m=sp.symbols('n N m',postive = True)
l = 3
u = 10
for p in range(5):
    print("p=",p)
    s=0
    for k_ in range(l,u+1):
        s += k_**p
    display(s==transfrom_sum(n**p,n,N,n_start=m).subs({N:u,m:l}).simplify())
    #display(transfrom_sum(n**p,n,N,n_start=m).subs({N:u,m:l}).simplify())